In [1]:
import numpy as np
from scipy.special import erf
import sys

In [2]:
# must-set-parameters for calculation
charge = 0

In [3]:
# just some book keeping
atomicNumber = {
    "H": 1,
    "He": 2,
    "Li": 3
}

In [4]:
def readXyzFile(fileName):
    f = open(fileName, "r")
    
    numAtoms = 0
    numAtomsCheck = 0
    coords = []
    atoms = []
    
    for idx, line in enumerate(f):
        # first line contains number of atoms
        if idx == 0:
            try:
                numAtoms = int(line.split()[0]) # ignore new line chars or whitespace
                continue
            except ValueError as e:
                print("Format error: No valid input for number of atoms given.")
                exit(1)
            except IndexError as e:
                print("Format error: First line is empty.")
        
        # second line is empty and can be skipped
        if idx == 1:
            continue
        
        # rest of lines contain atom type and xyz coords
        if idx > 1:
            numAtomsCheck += 1
            arr = line.split()
            
            atoms.append(arr[0]) 
            
            coords.append([
                float(arr[1]),
                float(arr[2]),
                float(arr[3])
            ])
            
    
    # additional format checks
    if numAtoms != numAtomsCheck or numAtoms != len(atoms):
        raise ValueError("Format error: Number of atoms in first line does not match number of entries.")
    
    # don't forget to close the file
    f.close()
    
    return numAtoms, np.asarray(coords), atoms

In [5]:
# all the integral functions are here

def productOfTwoGaussians(f1, f2):
    """pass exponent and center as tuple (SO pp.411)"""
    a, R_a = f1
    b, R_b = f2
    
    p = a + b
    norm = (4 * a * b / (np.pi**2))**0.75 # product of normalization constants of two 1s Gaussians
    R_p = (a * R_a + b * R_b) / (a + b)
    K = norm * np.exp(- a * b * np.linalg.norm(R_a - R_b)**2 / (a + b) )
    
    return p, R_p, K


def overlapIntegral(f1, f2):
    """pass exponent and center as tuple (SO pp.412)"""
    
    p, R_p, K = productOfTwoGaussians(f1, f2)
    return K * (np.pi / p)**1.5


def T_e(f1, f2):
    """pass exponent and center as tuple (SO pp.412)"""
    a, R_a = f1
    b, R_b = f2
    
    p, R_p, K = productOfTwoGaussians(f1, f2)
    prefactor = a * b / (a + b)
    return prefactor * (3 - 2*prefactor * np.linalg.norm(R_a - R_b)**2) * (np.pi / (a + b))**1.5 * K


def F0(t):
    if t == 0:
        return 1
    else:
        return 0.5 * (np.pi * t)**0.5 * erf(t**0.5)
        


def V_eN(f1, f2, atomIdx):
    """pass exponent and center as tuple (SO pp.415)"""
    
    p, R_p, K = productOfTwoGaussians(f1, f2)
    
    
    R_c = coords[atomIdx] # center of atom in question
    atomType = atoms[atomIdx] # only for clarity, needed for atomCharge
    atomCharge = atomicNumber[atomType] # charge of atom (Z)
    
    valF0 = F0(p * np.linalg.norm(R_p - R_c)**2)
    
    return - 2 * np.pi / p * atomCharge * K * valF0


def V_ee(fA, fB, fC, fD):
    
    p, R_p, K_AB = productOfTwoGaussians(fA, fB)
    q, R_q, K_CD = productOfTwoGaussians(fC, fD)

    prefactor = 2*np.pi**2.5 / (p / q * (p + q)**0.5)
    valF0 = F0(p * q / (p + q) * np.linalg.norm(R_p - R_q)**2)
    return prefactor * K_AB * K_CD * valF0

In [7]:
# actual calculation starts here with reading the atoms and coordinates
numAtoms, coords, atoms = readXyzFile("data/h2.xyz")


# get number of electrons
N = charge
for atom in atoms:
    N += atomicNumber[atom]

    
# get size of basis
sizeBasis = numAtoms # only for STO-nG!!!
    
    
# initialize matrices
S = np.zeros((sizeBasis, sizeBasis)) # overlap
T = np.zeros((sizeBasis, sizeBasis))
V = np.zeros((sizeBasis, sizeBasis))


# iteration over atoms
for idx, val in enumerate(atoms):
    # charge and center of atom
    Z = atomicNumber[val]
    R = coords[idx]
    
    print("idx: {}, type: {}, Z = {}, coords: {}".format(idx, val, Z, R))
    

idx: 0, type: H, Z = 1, coords: [-1.54551  0.4996   0.     ]
idx: 1, type: H, Z = 1, coords: [-0.83751  0.4996   0.     ]


In [ ]:
class primGauss:
    def __init__(self, alpha, coeff, coords, l1 = 0, l2 = 0, l3 = 0):
        self.alpha = alpha
        self.coeff = coeff
        self.coords = coords
        self.norm = (2.0 * alpha / np.pi) ** 0.75 # + other terms if angular momentum != 0

In [ ]:
h1_a = primGauss(0.3425250914E+01, 0.1543289673E+00, [0, 0, 0])
h1_b = primGauss(0.6239137298E+00, 0.5353281423E+00, [0, 0, 0])
h1_c = primGauss(0.1688554040E+00, 0.4446345422E+00, [0, 0, 0])

h2_a = primGauss(0.3425250914E+01, 0.1543289673E+00, [1.2, 0, 0])
h2_b = primGauss(0.6239137298E+00, 0.5353281423E+00, [1.2, 0, 0])
h2_c = primGauss(0.1688554040E+00, 0.4446345422E+00, [1.2, 0, 0])

h1 = [h1_a, h1_b, h1_c]
h2 = [h2_a, h2_b, h2_c]

molecule = [h1, h2]

In [ ]:
def overlap(molecule):
    nAtoms = len(molecule)
    
    S = np.zeros([nAtoms, nAtoms])
    
    for i in range(nAtoms): # loop over rows of S
        for j in range(nAtoms): # loop over cols of S
            numPrimitves = len(molecule[i])
        
overlap(molecule)